# Database Read and Write Examples

## Step 1: Read data from a database table

**In CPD v4.x, there are two options for the insert to code feature. One uses the Flight service, the other uses Python database API, eg. for Db2 on Cloud, it uses the ibm_db and ibm_db_dbi.**

**The following 2 cells were generated by Insert to Code(pandas data frame) menu. You can either regenerate it for Connected Data defined in your project or modify this code.**

If you are not regenerating code, make sure to verify or change:

1. Connection name
2. Schema name
3. Table name

*Look for "To Do" tag in the code*

In [ ]:
import itc_utils.flight_service as itcfs

readClient = itcfs.get_flight_client()

DB2_Dev_data_request = {
    'connected_data_name': """CustomerChurnTable"""
}

flightInfo = itcfs.get_flight_info(readClient, nb_data_request=DB2_Dev_data_request)

data_df_1 = itcfs.read_pandas_and_concat(readClient, flightInfo)
data_df_1.head(10)


In [ ]:
# @hidden_cell

from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

DB2_Dev_metadata = wslib.get_connection("DB2_Dev")

import os, ibm_db, ibm_db_dbi as dbi, pandas as pd

DB2_Dev_dsn = 'DATABASE={};HOSTNAME={};PORT={};PROTOCOL=TCPIP;UID={uid};PWD={pwd};SECURITY=SSL'.format(
    DB2_Dev_metadata['database'],
    DB2_Dev_metadata['host'],
    DB2_Dev_metadata.get('port', 50000),
    uid=DB2_Dev_metadata['username'],
    pwd=DB2_Dev_metadata['password']
)

DB2_Dev_connection = dbi.connect(DB2_Dev_dsn)
   
# NOTE:
#  A row limit has been applied to the query to enable sample previewing.
#  Adjust the display message and query as needed by editing the following lines:
from IPython.core.display import display, HTML
display(HTML("A row limit of 5000 has been applied to the query to enable sample previewing. If the data set is larger, only the first 5000 rows will be loaded."))
query = 'SELECT * FROM "JLD84201"."CUSTOMER_CHURN" FETCH FIRST 5000 ROWS ONLY'

data_df_2 = pd.read_sql_query(query, con=DB2_Dev_connection)
data_df_2.head()

# After use, close the database connection with the following code:
# DB2_Dev_connection.close()


## Step 2: Use the connection object ##
Once we have the connection object, we can run any query. In this cell we retrieve data from a different table. You can try running any SQL query for tables in the same schema. 

In [ ]:
# Once we have the connection object, we can run any query. In this cell we retrieve data from a different table
query = 'SELECT * FROM "JLD84201"."MORTGAGE_APPLICANT"'
mortgageDF = pd.read_sql_query(query, con=DB2_Dev_connection)
mortgageDF.head()

## Step 3: Insert Data using JayDeBeApi

In [ ]:
# Let's use JayDeBeAPI to insert a row into a database
# Get the cursor object 
curs = DB2_Dev_connection.cursor()

In [ ]:
# Run the INSERT statement
curs.execute("INSERT INTO CUSTOMER_CHURN VALUES ('F','M',2,100000,'Y',37,2000,5,'CC',2,'K')")

In [ ]:
# Confirm that the row was inserted
curs.execute("SELECT * FROM CUSTOMER_CHURN WHERE CHURN = 'K'")
curs.fetchall()

## Step 4: Insert Data using SQLAlchemy API

The connection string for DB2/SQLAlchemy API has the following format: **db2+ibm_db://userid:password@db_url:db_port/db_name**

Example: 

- *db2+ibm_db://tlr28624:c%40kf12hbpw@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB*
- **If SSL checkbox in the connection is checked**: *db2+ibm_db://tlr28624:c%40kf12hbpw@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB;SECURITY=SSL;*

To avoid hardcoding userid and password in the code, you can use the **Insert Credentials** option to reference database userid and password. You can also reference the database name. 

In [ ]:
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

Customer_Churn_Table_credentials = wslib.get_connection("DB2_Dev")

db_username=Customer_Churn_Table_credentials['username']
db_password=Customer_Churn_Table_credentials['password']
db_name=Customer_Churn_Table_credentials['database']

In [ ]:
# Concatenate the connection string
connection_string = 'db2+ibm_db://' + db_username + ':' + db_password + '@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/' + db_name + ';SECURITY=SSL;'

In [ ]:
print(connection_string)

In [ ]:
import sqlalchemy

db = sqlalchemy.create_engine(connection_string)
# When we use the SQLAlchemy API, we need to use the engine object
engine = db.connect()
meta = sqlalchemy.MetaData(engine)

In [ ]:
# Insert a row
resultProxy = engine.execute("""INSERT INTO CUSTOMER_CHURN VALUES ('F','M',2,100000,'Y',37,2000,5,'CC',2,'S')""")

In [ ]:
query = "SELECT * FROM CUSTOMER_CHURN WHERE CHURN = 'S'"

results = engine.execute(query)

#res.fetchone()
#res.fetchmany(3)
results.fetchall()

## Step 5: Write a Pandas DataFrame to a Table

In some cases we may want to write an entire data frame to the database. In this example we create a simple pandas data frame and using the combination of SQLAlchemy API and pandas, we can write it to database. 

In [ ]:
# Define required libraries
import numpy as np
import pandas as pd

# Create a dataframe called "random_data" - 4 cols x 100 row 
random_data = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))
random_data.head()

In [ ]:
# Write data frame to a new table
# The engine object was created earlier with SQLAlchemy API
random_data.to_sql("PANDAS_TEST1",engine,if_exists='append')

In [ ]:
query = 'SELECT * FROM "JLD84201"."PANDAS_TEST1"'
testDF = pd.read_sql_query(query, con=DB2_Dev_connection)
testDF.head()

In [ ]:
DB2_Dev_connection.close()

**Written by Elena Lowery, elowery@us.ibm.com**